In [ ]:
!pip install darts
!pip install dask[dataframe]

In [ ]:
!pip uninstall numpy
!pip install numpy
!pip install --force-reinstall darts

In [ ]:
import pandas as pd
import numpy as np
import darts
from darts import TimeSeries
import os
from scipy.special import comb
import matplotlib.pyplot as plt
import darts.models.forecasting
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/weekly_summary_1991-2021.csv')
print(df.head())

In [ ]:
# Creating train-test split
col_index = 1
lags = 2
colnames_dat = df.columns
dat = np.array(df[colnames_dat[col_index]]).reshape(-1, 1)
# split_point = int(5/6 * len(dat))
# # Split into train, validation, and test sets
# tr_dat, ts_dat = dat[:split_point], dat[split_point:]
# fore_hor = len(ts_dat)
hor = 53
split_point = len(dat) - hor
# Split into train, validation, and test sets
tr_dat, ts_dat = dat[:split_point], dat[split_point:]
fore_hor = len(ts_dat)

In [ ]:
# Convert to Darts TimeSeries format
train_series = TimeSeries.from_values(tr_dat)
test_series = TimeSeries.from_values(ts_dat)

In [ ]:
# Create a folder by Equation name
folder_name = 'One_Step_Ahead_' + str(lags)
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"Folder '{folder_name}' created successfully.")
else:
    print(f"Folder '{folder_name}' already exists.")

Folder 'One_Step_Ahead_2' created successfully.


In [ ]:
from darts.metrics import metrics
smape_metric = {}
mase_metric = {}
mae_metric = {}
rmse_metric = {}
marre_metric = {}
model_train_pred_time = {}

## NBeats

In [ ]:
# Define the N-BEATS Model
model_name = 'NBEATSModel'
model = darts.models.forecasting.nbeats.NBEATSModel(input_chunk_length=lags, output_chunk_length=1, random_state=42)
start_time = time.time()
# Fit the model
model.fit(train_series)
# Perform recursive forecasting for fore_hor steps
forecast = []
current_input = train_series[-lags:]  # Start with the last lags points
for i in range(fore_hor):
  pred = model.predict(n=1, series=current_input)
  forecast.append(pred.values()[0, 0])
  current_input = TimeSeries.from_values(np.vstack((current_input.values()[1:], ts_dat[i])))
end_time = time.time()
execution_time = end_time - start_time
model_train_pred_time[model_name] = execution_time
smape_metric[model_name] = metrics.smape(test_series, TimeSeries.from_values(np.array(forecast)))
#mase_metric[model_name] = metrics.mase(test_series, TimeSeries.from_values(np.array(forecast)))
mae_metric[model_name] = metrics.mae(test_series, TimeSeries.from_values(np.array(forecast)))
rmse_metric[model_name] = metrics.rmse(test_series, TimeSeries.from_values(np.array(forecast)))
marre_metric[model_name] = metrics.marre(test_series, TimeSeries.from_values(np.array(forecast)))


In [ ]:
forecast_df = pd.DataFrame({model_name: forecast})
file_name = model_name + '.xlsx'
forecast_file_path = os.path.join(folder_name, file_name)
forecast_df.to_excel(forecast_file_path, index=False)
print(f"Forecast saved to {forecast_file_path}")

## LSTM

In [ ]:
# Define the LSTM Model
model_name = 'LSTM'
model = darts.models.forecasting.rnn_model.RNNModel(input_chunk_length = lags,
                                                    model='LSTM', random_state=42, training_length= lags+1)
# Fit the model
start_time = time.time()
model.fit(train_series)
# Perform recursive forecasting for fore_hor steps
forecast = []
current_input = train_series[-lags:]  # Start with the last lags points
for i in range(fore_hor):
  pred = model.predict(n=1, series=current_input)
  forecast.append(pred.values()[0, 0])
  current_input = TimeSeries.from_values(np.vstack((current_input.values()[1:], ts_dat[i])))

end_time = time.time()
execution_time = end_time - start_time
model_train_pred_time[model_name] = execution_time
smape_metric[model_name] = metrics.smape(test_series, TimeSeries.from_values(np.array(forecast)))
# mase_metric[model_name] = metrics.mase(test_series, TimeSeries.from_values(np.array(forecast)))
mae_metric[model_name] = metrics.mae(test_series, TimeSeries.from_values(np.array(forecast)))
rmse_metric[model_name] = metrics.rmse(test_series, TimeSeries.from_values(np.array(forecast)))
marre_metric[model_name] = metrics.marre(test_series, TimeSeries.from_values(np.array(forecast)))

In [ ]:
forecast_df = pd.DataFrame({model_name: forecast})
file_name = model_name + '.xlsx'
forecast_file_path = os.path.join(folder_name, file_name)
forecast_df.to_excel(forecast_file_path, index=False)
print(f"Forecast saved to {forecast_file_path}")

## NHits

In [ ]:
# Define the NHiTSModel Model
model_name = 'NHiTSModel'
model = darts.models.forecasting.nhits.NHiTSModel(input_chunk_length = lags, output_chunk_length = 1, random_state=42)
# Fit the model
start_time = time.time()
model.fit(train_series)
# Perform recursive forecasting for fore_hor steps
forecast = []
current_input = train_series[-lags:]  # Start with the last lags points
for i in range(fore_hor):
  pred = model.predict(n=1, series=current_input)
  forecast.append(pred.values()[0, 0])
  current_input = TimeSeries.from_values(np.vstack((current_input.values()[1:], ts_dat[i])))

end_time = time.time()
execution_time = end_time - start_time
model_train_pred_time[model_name] = execution_time
smape_metric[model_name] = metrics.smape(test_series, TimeSeries.from_values(np.array(forecast)))
# mase_metric[model_name] = metrics.mase(test_series, TimeSeries.from_values(np.array(forecast)))
mae_metric[model_name] = metrics.mae(test_series, TimeSeries.from_values(np.array(forecast)))
rmse_metric[model_name] = metrics.rmse(test_series, TimeSeries.from_values(np.array(forecast)))
marre_metric[model_name] = metrics.marre(test_series, TimeSeries.from_values(np.array(forecast)))

In [ ]:
forecast_df = pd.DataFrame({model_name: forecast})
file_name = model_name + '.xlsx'
forecast_file_path = os.path.join(folder_name, file_name)
forecast_df.to_excel(forecast_file_path, index=False)
print(f"Forecast saved to {forecast_file_path}")

## TransformerModel

In [ ]:
# Define the TransformerModel Model
model_name = 'TransformerModel'
model = darts.models.forecasting.transformer_model.TransformerModel(input_chunk_length = lags, output_chunk_length = 1, random_state=42)
# Fit the model
start_time = time.time()
model.fit(train_series)
# Perform recursive forecasting for fore_hor steps
forecast = []
current_input = train_series[-lags:]  # Start with the last lags points
for i in range(fore_hor):
  pred = model.predict(n=1, series=current_input)
  forecast.append(pred.values()[0, 0])
  current_input = TimeSeries.from_values(np.vstack((current_input.values()[1:], ts_dat[i])))

end_time = time.time()
execution_time = end_time - start_time
model_train_pred_time[model_name] = execution_time
smape_metric[model_name] = metrics.smape(test_series, TimeSeries.from_values(np.array(forecast)))
# mase_metric[model_name] = metrics.mase(test_series, TimeSeries.from_values(np.array(forecast)))
mae_metric[model_name] = metrics.mae(test_series, TimeSeries.from_values(np.array(forecast)))
rmse_metric[model_name] = metrics.rmse(test_series, TimeSeries.from_values(np.array(forecast)))
marre_metric[model_name] = metrics.marre(test_series, TimeSeries.from_values(np.array(forecast)))

In [ ]:
forecast_df = pd.DataFrame({model_name: forecast})
file_name = model_name + '.xlsx'
forecast_file_path = os.path.join(folder_name, file_name)
forecast_df.to_excel(forecast_file_path, index=False)
print(f"Forecast saved to {forecast_file_path}")

Forecast saved to One_Step_Ahead_2/TransformerModel.xlsx


## DLinearModel

In [ ]:
# Define the DLinearModel Model
model_name = 'DLinearModel'
model = darts.models.forecasting.dlinear.DLinearModel(input_chunk_length = lags, output_chunk_length = 1, random_state=42)
# Fit the model
start_time = time.time()
model.fit(train_series)
# Perform recursive forecasting for fore_hor steps
forecast = []
current_input = train_series[-lags:]  # Start with the last lags points
for i in range(fore_hor):
  pred = model.predict(n=1, series=current_input)
  forecast.append(pred.values()[0, 0])
  current_input = TimeSeries.from_values(np.vstack((current_input.values()[1:], ts_dat[i])))

end_time = time.time()
execution_time = end_time - start_time
model_train_pred_time[model_name] = execution_time
smape_metric[model_name] = metrics.smape(test_series, TimeSeries.from_values(np.array(forecast)))
# mase_metric[model_name] = metrics.mase(test_series, TimeSeries.from_values(np.array(forecast)))
mae_metric[model_name] = metrics.mae(test_series, TimeSeries.from_values(np.array(forecast)))
rmse_metric[model_name] = metrics.rmse(test_series, TimeSeries.from_values(np.array(forecast)))
marre_metric[model_name] = metrics.marre(test_series, TimeSeries.from_values(np.array(forecast)))

In [ ]:
forecast_df = pd.DataFrame({model_name: forecast})
file_name = model_name + '.xlsx'
forecast_file_path = os.path.join(folder_name, file_name)
forecast_df.to_excel(forecast_file_path, index=False)
print(f"Forecast saved to {forecast_file_path}")

Forecast saved to One_Step_Ahead_2/DLinearModel.xlsx


## NLinearModel

In [ ]:
# Define the NLinearModel Model
model_name = 'NLinearModel'
model = darts.models.forecasting.nlinear.NLinearModel(input_chunk_length = lags, output_chunk_length = 1, random_state=42)
# Fit the model
start_time = time.time()
model.fit(train_series)
# Perform recursive forecasting for fore_hor steps
forecast = []
current_input = train_series[-lags:]  # Start with the last lags points
for i in range(fore_hor):
  pred = model.predict(n=1, series=current_input)
  forecast.append(pred.values()[0, 0])
  current_input = TimeSeries.from_values(np.vstack((current_input.values()[1:], ts_dat[i])))

end_time = time.time()
execution_time = end_time - start_time
model_train_pred_time[model_name] = execution_time
smape_metric[model_name] = metrics.smape(test_series, TimeSeries.from_values(np.array(forecast)))
# mase_metric[model_name] = metrics.mase(test_series, TimeSeries.from_values(np.array(forecast)))
mae_metric[model_name] = metrics.mae(test_series, TimeSeries.from_values(np.array(forecast)))
rmse_metric[model_name] = metrics.rmse(test_series, TimeSeries.from_values(np.array(forecast)))
marre_metric[model_name] = metrics.marre(test_series, TimeSeries.from_values(np.array(forecast)))

In [ ]:
forecast_df = pd.DataFrame({model_name: forecast})
file_name = model_name + '.xlsx'
forecast_file_path = os.path.join(folder_name, file_name)
forecast_df.to_excel(forecast_file_path, index=False)
print(f"Forecast saved to {forecast_file_path}")

Forecast saved to One_Step_Ahead_2/NLinearModel.xlsx


## TiDEModel

In [ ]:
# Define the TiDEModel Model
model_name = 'TiDEModel'
model = darts.models.forecasting.tide_model.TiDEModel(input_chunk_length = lags, output_chunk_length = 1, random_state=42)
# Fit the model
start_time = time.time()
model.fit(train_series)
# Perform recursive forecasting for fore_hor steps
forecast = []
current_input = train_series[-lags:]  # Start with the last lags points
for i in range(fore_hor):
  pred = model.predict(n=1, series=current_input)
  forecast.append(pred.values()[0, 0])
  current_input = TimeSeries.from_values(np.vstack((current_input.values()[1:], ts_dat[i])))

end_time = time.time()
execution_time = end_time - start_time
model_train_pred_time[model_name] = execution_time
smape_metric[model_name] = metrics.smape(test_series, TimeSeries.from_values(np.array(forecast)))
# mase_metric[model_name] = metrics.mase(test_series, TimeSeries.from_values(np.array(forecast)))
mae_metric[model_name] = metrics.mae(test_series, TimeSeries.from_values(np.array(forecast)))
rmse_metric[model_name] = metrics.rmse(test_series, TimeSeries.from_values(np.array(forecast)))
marre_metric[model_name] = metrics.marre(test_series, TimeSeries.from_values(np.array(forecast)))

In [ ]:
forecast_df = pd.DataFrame({model_name: forecast})
file_name = model_name + '.xlsx'
forecast_file_path = os.path.join(folder_name, file_name)
forecast_df.to_excel(forecast_file_path, index=False)
print(f"Forecast saved to {forecast_file_path}")

Forecast saved to One_Step_Ahead_2/TiDEModel.xlsx


## XGBModel

In [ ]:
# Define the XGBModel Model
model_name = 'XGBModel'
model = darts.models.forecasting.xgboost.XGBModel(lags = lags, output_chunk_length = 1, random_state=42)
# Fit the model
start_time = time.time()
model.fit(train_series)
# Perform recursive forecasting for fore_hor steps
forecast = []
current_input = train_series[-lags:]  # Start with the last lags points
for i in range(fore_hor):
  pred = model.predict(n=1, series=current_input)
  forecast.append(pred.values()[0, 0])
  current_input = TimeSeries.from_values(np.vstack((current_input.values()[1:], ts_dat[i])))

end_time = time.time()
execution_time = end_time - start_time
model_train_pred_time[model_name] = execution_time
smape_metric[model_name] = metrics.smape(test_series, TimeSeries.from_values(np.array(forecast)))
# mase_metric[model_name] = metrics.mase(test_series, TimeSeries.from_values(np.array(forecast)))
mae_metric[model_name] = metrics.mae(test_series, TimeSeries.from_values(np.array(forecast)))
rmse_metric[model_name] = metrics.rmse(test_series, TimeSeries.from_values(np.array(forecast)))
marre_metric[model_name] = metrics.marre(test_series, TimeSeries.from_values(np.array(forecast)))

In [ ]:
forecast_df = pd.DataFrame({model_name: forecast})
file_name = model_name + '.xlsx'
forecast_file_path = os.path.join(folder_name, file_name)
forecast_df.to_excel(forecast_file_path, index=False)
print(f"Forecast saved to {forecast_file_path}")

## RandomForest

In [ ]:
# Define the RandomForest Model
model_name = 'RandomForest'
model = darts.models.forecasting.random_forest.RandomForest(lags = lags, output_chunk_length = 1, random_state=42)
# Fit the model
start_time = time.time()
model.fit(train_series)
# Perform recursive forecasting for fore_hor steps
forecast = []
current_input = train_series[-lags:]  # Start with the last lags points
for i in range(fore_hor):
  pred = model.predict(n=1, series=current_input)
  forecast.append(pred.values()[0, 0])
  current_input = TimeSeries.from_values(np.vstack((current_input.values()[1:], ts_dat[i])))

end_time = time.time()
execution_time = end_time - start_time
model_train_pred_time[model_name] = execution_time
smape_metric[model_name] = metrics.smape(test_series, TimeSeries.from_values(np.array(forecast)))
# mase_metric[model_name] = metrics.mase(test_series, TimeSeries.from_values(np.array(forecast)))
mae_metric[model_name] = metrics.mae(test_series, TimeSeries.from_values(np.array(forecast)))
rmse_metric[model_name] = metrics.rmse(test_series, TimeSeries.from_values(np.array(forecast)))
marre_metric[model_name] = metrics.marre(test_series, TimeSeries.from_values(np.array(forecast)))

In [ ]:
forecast_df = pd.DataFrame({model_name: forecast})
file_name = model_name + '.xlsx'
forecast_file_path = os.path.join(folder_name, file_name)
forecast_df.to_excel(forecast_file_path, index=False)
print(f"Forecast saved to {forecast_file_path}")

Forecast saved to One_Step_Ahead_2/RandomForest.xlsx


In [ ]:
smape_metrics_df = pd.DataFrame.from_dict(smape_metric, orient='index', columns=['SMAPE'])
rmse_metrics_df = pd.DataFrame.from_dict(rmse_metric, orient='index', columns=['RMSE'])
mae_metrics_df = pd.DataFrame.from_dict(mae_metric, orient='index', columns=['MAE'])
marre_metrics_df = pd.DataFrame.from_dict(marre_metric, orient='index', columns=['MARRE'])
execution_time_df = pd.DataFrame.from_dict(model_train_pred_time, orient='index', columns=['Time'])

In [ ]:
merged_df = pd.concat([smape_metrics_df, rmse_metrics_df, mae_metrics_df, marre_metrics_df, execution_time_df], axis=1)
merged_df